<a href="https://colab.research.google.com/github/johanhoffman/DD2363_VT21/blob/leobergman/LeoBergman_Lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab 3: Iterative methods**
**Leo Bergman**

# **Abstract**

In this lab I performed some experiments with iterative methods to solve equation systems Ax=b as well as to approximate roots for given non linear functions.

#**About the code**

A short statement on who is the author of the file, and if the code is distributed under a certain license. 

In [1]:
"""This program is a template for lab reports in the course"""
"""DD2363 Methods in Scientific Computing, """
"""KTH Royal Institute of Technology, Stockholm, Sweden."""

# Copyright (C) 2021 Leo Bergman (bergmanleo@gmail.com)
# Template by Johan Hoffman (jhoffman@kth.se)

# This file is part of the course DD2365 Advanced Computation in Fluid Mechanics
# KTH Royal Institute of Technology, Stockholm, Sweden
#
# This is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

'KTH Royal Institute of Technology, Stockholm, Sweden.'

# **Set up environment**

To have access to the neccessary modules you have to run this cell. If you need additional modules, this is where you add them. 

In [2]:
# Load neccessary modules.
from google.colab import files
from scipy.sparse import random
import time
import timeit
import numpy as np
import random
from matplotlib import pyplot as plt
from matplotlib import tri
from matplotlib import axes
from mpl_toolkits.mplot3d import Axes3D
import scipy.sparse as sparse
from scipy import stats
import scipy
import numpy as np
from numpy.linalg import inv,norm,solve,lstsq,det
import unittest
from functools import reduce
from numpy.polynomial import polynomial as Poly


# **Introduction**

In this lab I have tried out various fixed point methods such as GMRES, Jacob and Gauss-Seidel.


# **Method**

I have implemented the algorithms with inspiration from the course book by Professor Hoffman. Iterative methods from chapter 7,8.
GMRES and Arnoldi iteration are both krylov subspace methods. GMRES from algorithm 7.2.Arnoldi from algorithm 7.3.

Assuming here that the spectral radius of the matrix $M_j$ is less than one: $\rho(I-D^1 *A)<1$

Otherwise we're not assertain a convergence.

I should really test this out prior to run the Jacobi iteration.

Another fun fact:
The Jacobi iteration is a Richardson iteration when $\alpha= 1 $ 




In [3]:

class Itermethods:
  def __init__(self):
    pass

  @staticmethod
  def jacobiIteration(self,v,TOL=1e-6):
    n = self.shape[0]
    Dinv=np.zeros((n,n))
    x = np.random.rand(self.shape[0])
    for i in range(self.shape[0]):
      Dinv[i,i] = 1/self[i,i]
      
    M = np.identity(n)-np.dot(Dinv,self)

    while norm(np.dot(self,x)-v)>TOL:
      x = np.dot(M,x)+np.dot(Dinv,v)
    return x

  @staticmethod
  def gaussSeidel(self,v,TOL=1e-6):

    n = self.shape[0]
    astar = self.copy()
    for i in range(n):
      for j in range(n):
        if (j>i):
          astar[i,j]=0
    linv = inv(astar)
    A = np.identity(n)-np.dot(linv,self)
    c = np.dot(linv,v)

    x1 = np.random.rand(n)

    iter = 0
    r = np.ones
    while norm(np.dot(self,x1)-v)>TOL:
      x1 = np.dot(A,x1)+c

      iter+=1
    return x1


  @staticmethod
  def gmres(self,v,TOL=1e-6):
    Q = np.zeros(self.shape)
    Q[:,0] = v
    k = 0
    iter = 0
    r = v
    while norm(r)/norm(v)>TOL:
      #e_2 = np.eye(1,k+1,0)
      e_1 = np.zeros(k+1)
      e_1[0] = 1
      Q,H = Itermethods.arnoldi_iteration(self,v,k)
     
      y = lstsq(H,norm(v)*e_1,rcond=None)[0]
      r= H.dot(y)
      r = norm(v)*e_1-r
      iter+=1
      k+=1

    x = Q[:,0:k-1].dot(y)
    return x

  @staticmethod
  def arnoldi_iteration(A,b,k):
    Q = np.zeros((A.shape[0],k+1))
    Q[:,0] = b/norm(b)
    H = np.zeros((k+1,k))
    v = np.zeros(A.shape[0])
    for j in range(k):
      v = A.dot(Q[:,j])
      for i in range(j+1):
        H[i,j] = np.dot(Q[:,i].conj(),v)
        v = v-H[i,j]*Q[:,i]
      H[j+1,j] = norm(v)
      if H[j+1,j] <1e-10:
        return Q,H
      Q[:,j+1] = v/H[j+1,j]
    return Q,H

  @staticmethod
  def newtonsMethod(self,x_0,dx=1e-11,TOL=1e-10):
    x = x_0
    iter = 0
    while abs(self(x)>TOL):
      df = (self(x+dx)-self(x-dx))/(2*dx)
      if np.allclose(df, 0, atol=1e-10):
        raise Exception("The Jacobian is getting singular @ iter: " + str(iter))
      iter+=1
      x = x-self(x)/df
    return x
 




# **Results**

**Proof of correctness**

In [4]:

class Test(unittest.TestCase):
 
  
  def testJacobi(self):
    n = np.random.randint(1,20)
    M = np.random.rand(n,n)
    v = np.random.rand(n)
    alpha = 1
    B = inv(M)*alpha
    C = np.dot(M,B)
    y = np.random.rand(n)
    bb = np.dot(C,y)
    approx = Itermethods.jacobiIteration(C,bb)
    exact = solve(C,bb)
    gs = np.dot(C,approx)
    np.testing.assert_almost_equal(norm(approx-exact),0,decimal=7)
    np.testing.assert_almost_equal(norm(gs-bb),0,decimal=7)


  def testGaussSeidel(self):
    n = np.random.randint(1,20)
    M = np.random.rand(n,n)
    v = np.random.rand(n)
    alpha = 1
    B = inv(M)*alpha
    C = np.dot(M,B)
    y = np.random.rand(n)
    bb = np.dot(C,y)
    approx = Itermethods.gaussSeidel(C,bb)
    exact = solve(C,bb)
    gs = np.dot(C,approx)
    np.testing.assert_almost_equal(norm(approx-exact),0,decimal=7)
    np.testing.assert_almost_equal(norm(gs-bb),0,decimal=7)

  def testNewton(self):
    n = random.randint(1,10)
    zed = np.random.rand(n)
    polly = [(zed[i],1) for i in range(n)]
    polly2 = reduce(Poly.polymul,polly)
    f = lambda x: Poly.polyval(x, polly2)
    x1 = np.zeros(n,dtype = 'float64')
    xn = Itermethods.newtonsMethod(f,x_0 = np.random.rand())

    np.testing.assert_almost_equal(f(xn),0,5)

  def testGMRES(self):
    n = np.random.randint(1,20)
    M = np.random.rand(n,n)
    v = np.random.rand(n)
    B = inv(M)
    C = np.dot(M,B)
    y = np.random.rand(n)
    bb = np.dot(C,y)
    approx = Itermethods.gmres(C,bb)
    exact = solve(C,bb)
    gmres = np.dot(C,approx)

    np.testing.assert_almost_equal(norm(approx-exact),0,decimal=7)
    np.testing.assert_almost_equal(norm(gmres-bb),0,decimal=7)


if __name__ == '__main__':
  unittest.main(argv=['first-arg-is-ignored'],exit = False)

....
----------------------------------------------------------------------
Ran 4 tests in 0.045s

OK


**Performance test**

The iterative methods worked as expected. It was really interesting to discover the Krylov spaces.

# **Discussion**

It would be very interesting to plot those functions to compare the convergence. My hypothesis is that GMRES is very fast in convergence.